In [1]:
%matplotlib inline


In [2]:
import tensorflow as tf
import numpy as np
import cv2 
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import time


model = tf.saved_model.load("./exported_model")
model_fn = model.signatures['serving_default']
tracker = DeepSort(max_age=5,max_iou_distance=0.2,)

2025-02-28 11:18:42.507666: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-28 11:18:42.518102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740716322.530084  295672 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740716322.533461  295672 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 11:18:42.546068: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
(INPUT_WIDTH, INPUT_HEIGHT) = (256, 256) 
SCORE_THRESHOLD = 0.3
def detect_objects(image):

    resized_image = cv2.resize(src=image, dsize=(INPUT_WIDTH,INPUT_HEIGHT))
    input_tensor = tf.convert_to_tensor(resized_image)
    input_tensor = input_tensor[tf.newaxis,...]
    detections = model_fn(input_tensor)

    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

    return boxes, classes, scores, resized_image

def convert_to_deepsort_format(boxes, classes, scores):
    objects = []
    for i in range(len(boxes)):
        if classes[i] in [1]: 
            x_min, y_min, x_max, y_max = boxes[i]
            score = scores[i]
            if score > SCORE_THRESHOLD:
                left = x_min
                top = y_min
                width = x_max - x_min
                height = y_max - y_min
                objects.append(([left, top, width, height], float(score), 1))

    return objects


In [4]:
# Define a function to draw bounding boxes
original_width, original_height = 256, 256

# New image size
new_width, new_height = 1280, 720
scale_x = new_width / original_width
scale_y = new_height / original_height
def draw_boxes_track(image, tracks):
    global scale_x, scale_y
    for track in tracks:
        y_min, x_min, y_max, x_max = track.to_tlbr().astype(int)
        y_min = int(y_min*scale_y)
        x_min = int(x_min*scale_x)
        y_max = int(y_max*scale_y)
        x_max = int(x_max*scale_x)
        track_id = track.track_id
      
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
      
        label = f"ID: {track_id}"
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
    return image
def draw_boxes_detect(image, boxes, scores, score_threshold):
    for i in range(len(boxes)):
        y_min, x_min, y_max, x_max = boxes[i].astype(int)
        if scores[i] <= score_threshold:
            continue
      
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 1)
      
        label = f"Car"
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
    return image

In [8]:
cap = cv2.VideoCapture("/home/giang/Downloads/test2.mp4")

In [9]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter("output1.mp4", fourcc, 24, (frame_width * 2, frame_height))
while cap.isOpened():
    ret, frame = cap.read()
    if ret is False:
        break
    origin_frame = frame.copy()
    # origin_frame= cv2.cvtColor(origin_frame, cv2.COLOR_BGR2RGB)
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, classes, scores, resized_image = detect_objects(frame)
    objects = convert_to_deepsort_format(boxes, classes, scores)
    if len(objects) > 0:
        tracks = tracker.update_tracks(objects, frame=resized_image)
    else:
        tracks = []

    
    outputImage = draw_boxes_track(frame, tracks)
    concatenated_frame = cv2.hconcat([origin_frame, outputImage])
    out.write(concatenated_frame)
    # plt.imshow(concatenated_frame)
    # plt.show()
    
    # clear_output(wait=True)  
    
    # time.sleep(0.1)

    

In [10]:
cap.release()
out.release()
import os
print(os.getcwd())

/home/giang/vehicle_detection
